<a href="https://colab.research.google.com/github/YorkJong/vistock/blob/feature%2Fibd/notebooks/ibd_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install vistock from GitHub (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

In [3]:
%pip install "git+https://github.com/YorkJong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/YorkJong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-1n_ejeqg
  Running command git clone --filter=blob:none --quiet https://github.com/YorkJong/vistock.git /tmp/pip-req-build-1n_ejeqg
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/YorkJong/vistock.git to commit e00628b830361c0021df97305d3e06f5f7b56345
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.1 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.4.0-py3-none-any.whl size=42026 sha256=e35919b882b565f434ca5bfd3332d3e1d4e9963e790102a0234247d4d909facd
  Stored in directory: /tmp/pip-ephem-wheel-cache-m7ykeati/wheels/3e/1c/74/97ec620813c1bb106679625869f7409ca5be8735d0a74457bf
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4

### Setup and Configuration

In [14]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [15]:
# install cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [16]:
import os
from datetime import datetime

from vistock import ibd
from vistock import tw
from vistock.stock_indices import get_tickers


def rank(code, min_percentile=90, period='2y',
         tickers_getter=get_tickers, ref_ticker='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)
    rank_stock, rank_indust = ibd.rankings(tickers, period=period,
                                           ref_ticker=ref_ticker)

    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    print('\nStock Rankings:')
    #display(rank_stock)
    top_stocks = rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile]
    display(top_stocks)
    num_rows, _ = top_stocks.shape
    print(f'\nnumber of filtered tickers: {num_rows}')
    top_stock_list = list(top_stocks["Ticker"])
    if tickers_getter is tw.get_tickers:
        top_stock_list = [tw.stock_name(ticker) for ticker in top_stock_list]
    print(top_stock_list)

    print('\n\nIndustry Rankings:')
    top_industries = rank_indust[rank_indust[ibd.TITLE_PERCENTILE] >= min_percentile]
    display(top_industries)
    num_rows, _ = top_industries.shape
    print(f'\nnumber of filtered sectors: {num_rows}')

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Save to CSV
    print("\n\n***")
    today = datetime.now().strftime('%Y%m%d')
    for table, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_{today}.csv'
        table.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock, rank_indust


### Usage Explanation

##### Parameters
source (選擇要分析的股票來源):
- The source of stocks to analyze

min_percentile (最小百分位)
- The minimum percentile for a stock to be included in the rankings.

period (歷史資料時間範圍)：
- The period for which to fetch historical data.

### IBD RS Rating and Ranking

In [17]:
source = "All Indices" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "PHLX Semiconductor", "All Indices"]
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'SPX+DJIA+NDX+SOX',
}

rank_stock, rank_indust = rank(code_from_name[source], min_percentile, period)

ERROR:yfinance:$BF.B: possibly delisted; No price data found  (period=2y)


$BF.B: possibly delisted; No price data found  (period=2y)


ERROR:yfinance:ARM: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:SW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:SOLV: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:GEV: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:VLTO: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:BRK.B: No data found, symbol may be delisted



Stock Rankings:


,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
315,VST,Utilities,Utilities - Independent Power Producers,159.35,192.00,238.58,140.22,99,99,99,97,1
17,NVDA,Technology,Semiconductors,152.22,187.10,169.40,177.27,99,99,99,99,2
50,IRM,Real Estate,REIT - Specialty,144.26,124.08,116.53,103.70,99,95,88,69,3
62,FICO,Technology,Software - Application,142.48,131.59,119.32,131.11,99,97,91,96,4
389,GDDY,Technology,Software - Infrastructure,141.00,127.27,132.61,118.52,99,97,97,92,5
411,HWM,Industrials,Aerospace & Defense,136.11,121.73,134.66,114.15,99,94,98,88,6
103,MMM,Industrials,Conglomerates,135.53,104.54,112.07,81.53,98,80,83,13,7
198,AXON,Industrials,Aerospace & Defense,134.00,99.26,113.24,113.21,98,68,84,86,8
229,NRG,Utilities,Utilities - Independent Power Producers,133.17,135.64,171.52,116.56,98,98,99,90,9
484,TRGP,Energy,Oil & Gas Midstream,131.50,125.76,122.31,106.44,98,96,93,75,10



number of filtered tickers: 52
['VST', 'NVDA', 'IRM', 'FICO', 'GDDY', 'HWM', 'MMM', 'AXON', 'NRG', 'TRGP', 'UHS', 'KKR', 'GE', 'TSM', 'MHK', 'MPWR', 'ANET', 'ISRG', 'NTAP', 'AVGO', 'TYL', 'RCL', 'K', 'VTR', 'CBRE', 'EFX', 'BRO', 'CEG', 'WELL', 'COST', 'CFG', 'COHR', 'META', 'GRMN', 'CTAS', 'DHI', 'HCA', 'EXR', 'GS', 'MSI', 'TT', 'HIG', 'PGR', 'FITB', 'TMUS', 'GEN', 'IP', 'LLY', 'KLAC', 'RTX', 'PM', 'EBAY']


Industry Rankings:


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
74,Utilities - Independent Power Producers,Utilities,146.26,163.82,205.05,128.39,"VST,NRG",98,99,99,98,1
5,Medical Care Facilities,Healthcare,120.29,101.34,109.72,107.30,"UHS,HCA,DVA",97,89,92,83,2
19,Oil & Gas Midstream,Energy,116.74,108.45,111.29,96.99,"TRGP,KMI,WMB,OKE",96,96,95,49,3
49,Conglomerates,Industrials,115.52,99.05,102.60,85.44,"MMM,HON",95,75,67,12,4
64,REIT - Healthcare Facilities,Real Estate,114.97,99.87,99.82,87.93,"VTR,WELL,DOC",94,82,61,21,5
71,Tobacco,Consumer Defensive,113.92,100.10,98.45,84.59,"PM,MO",93,84,55,8,6
46,Residential Construction,Consumer Cyclical,112.52,100.24,107.74,111.51,"DHI,PHM,NVR,LEN",92,85,87,94,7
23,Computer Hardware,Technology,109.68,133.98,144.53,156.05,"ANET,NTAP,STX,HPQ,SMCI,WDC",91,98,98,99,8
79,Consulting Services,Industrials,109.24,99.39,96.46,104.24,"EFX,VRSK",90,78,43,73,9



number of filtered sectors: 9


***
Your "SPX+DJIA+NDX+SOX_stocks_2y_20240808.csv" is in the "out" folder.
Your "SPX+DJIA+NDX+SOX_industries_2y_20240808.csv" is in the "out" folder.
***



In [27]:
# @title Filtered Stocks with Increasing RS > 100
filtered_rank_stock = rank_stock[
    (rank_stock["Relative Strength"] > 100)
    & (rank_stock["Relative Strength"] > rank_stock["1 Month Ago"])
    & (rank_stock["1 Month Ago"] > rank_stock["3 Months Ago"])
    & (rank_stock["3 Months Ago"] > rank_stock["6 Months Ago"])
    #& (rank_stock["Percentile"] > 90)
]
display(filtered_rank_stock)

,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
50,IRM,Real Estate,REIT - Specialty,144.26,124.08,116.53,103.70,99,95,88,69,3
484,TRGP,Energy,Oil & Gas Midstream,131.50,125.76,122.31,106.44,98,96,93,75,10
504,TYL,Technology,Software - Application,120.65,108.40,102.57,99.21,96,87,65,57,21
152,VTR,Real Estate,REIT - Healthcare Facilities,119.46,101.50,96.67,85.34,95,74,46,20,24
171,WELL,Real Estate,REIT - Healthcare Facilities,117.52,103.73,103.56,100.23,94,78,68,60,29
164,EXR,Real Estate,REIT - Industrial,115.94,99.11,97.98,90.57,92,68,49,36,38
511,MSI,Technology,Communication Equipment,115.81,105.61,103.73,95.30,92,82,68,48,40
173,IP,Consumer Cyclical,Packaging & Containers,114.81,106.32,105.74,91.51,91,83,73,38,47
209,PM,Consumer Defensive,Tobacco,114.51,99.61,97.58,84.85,90,69,48,19,51
117,FSLR,Technology,Solar,114.43,114.00,96.42,79.16,89,92,44,8,53


#### IBD RS Rating and Ranking for Taiwan Stocks

In [35]:
from vistock import tw

source = "上市+上櫃" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]
min_percentile = 95 # @param {"type":"slider","min":1,"max":99,"step":1}
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

tw_stocks, tw_industries = rank(code_from_name[source], min_percentile, period,
     tickers_getter=tw.get_tickers, ref_ticker='^TWII')

ERROR:yfinance:00932.TW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:6658.TW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:6928.TW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:00949.TW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:020038.TW: Period '2y' is invalid, must be one of ['1d', '5d']
ERROR:yfinance:020018.TW: Period '2y' is invalid, must be one of ['1d', '5d']
ERROR:yfinance:020020.TW: Period '2y' is invalid, must be one of ['1d', '5d']
ERROR:yfinance:6949.TW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:6423.TW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:020037.TW: Period '2y' is invalid, must be one of ['1d', '5d']
ERROR:yfinance:


Stock Rankings:


,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
207,6144.TWO,Communication Services,Entertainment,470.91,254.34,119.00,83.36,99.0,99,87,15,1
1162,1799.TWO,Healthcare,Drug Manufacturers - Specialty & Generic,419.95,345.56,206.79,200.35,99.0,99,99,98,2
1347,8374.TW,Industrials,Industrial Distribution,357.29,265.46,83.28,83.66,99.0,99,38,16,3
1643,2365.TW,Technology,Computer Hardware,354.55,280.49,127.23,162.31,99.0,99,92,96,4
625,6442.TW,Technology,Electronic Components,352.21,409.84,189.05,137.16,99.0,99,98,92,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1076,1784.TWO,Healthcare,Diagnostics & Research,149.34,174.76,87.46,89.94,95.0,97,49,35,96
1961,6212.TWO,Real Estate,Real Estate - Development,148.56,111.69,119.20,97.20,95.0,83,87,57,97
1140,6171.TWO,Real Estate,Real Estate - Development,148.31,127.89,135.64,116.14,95.0,90,93,83,98
1443,9946.TW,Real Estate,Real Estate - Diversified,148.12,163.15,171.18,131.34,95.0,96,98,90,99



number of filtered tickers: 100
['得利影', '易威', '羅昇', '昆盈', '光聖', '穎漢', '均華', '皇昌', '新復興', '海悅', '欣巴巴', '晶彩科', '康全電訊', '所羅門', '福裕', '慧友', '弘憶股', '京城', '高鋒', '福大', '彬台', '新門', '世紀', '花王', '翔耀', '永信建', '訊舟', '天品', '福懋油', '全譜', '慶騰', '志聖', '均豪', '東捷', '喬福', '京晨科', '擎亞', '鑫科', '弘塑', '系微', '安國', '天方能源', '鑫龍騰', '泰偉', '三地開發', '順藥', '旺矽', '德晉', '聯上發', '聯鈞', '天揚', '鏵友益', '昇陽半導體', '昇益', '訊聯基因', '藥華藥', '華友聯', '錦明', '泰金-KY', '太普高', '和椿', '鈊象', '及成', '錸德', '宏碩系統', '藝舍-KY', '光明', '波力-KY', '富宇', '迎廣', '雲豹能源', '華義', '益登', '宏旭-KY', '華城', '森寶', '鈺邦', '天剛', '大飲', '盟立', '勝昱', '坤悅', '精湛', '峰源-KY', '佳能', '愛山林', '友威科', '一詮', '昇達科', '精材', '達能', '晶悅', '惠特', '加捷生醫', '上詮', '訊聯', '理銘', '大城地產', '三發地產', '合騏']


Industry Rankings:


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
93,Industrial Distribution,Industrials,157.61,132.06,85.54,89.49,"8374.TW,911608.TW,3114.TWO,2373.TW",98.0,98,31,23,1
1,Real Estate - Development,Real Estate,136.88,121.48,126.36,107.49,"2524.TW,5508.TWO,3188.TWO,2537.TW,5455.TWO,143...",97.0,96,99,89,2
29,Real Estate - Diversified,Real Estate,127.67,117.57,125.11,116.42,"1438.TW,9946.TW,6219.TWO,2520.TW,2545.TW,5512....",96.0,95,98,96,3
28,Utilities - Renewable,Utilities,120.37,130.55,105.97,109.59,"6869.TW,6873.TW,6806.TW,8087.TWO",95.0,97,92,90,4



number of filtered sectors: 4


***
Your "TWSE+TPEX_stocks_2y_20240808.csv" is in the "out" folder.
Your "TWSE+TPEX_industries_2y_20240808.csv" is in the "out" folder.
***



In [36]:
# @title Filtered Taiwan Stocks with Increasing RS > 100

def stock_ticker_to_name(df_stocks):
    # Iterate over the 'Ticker' column in the tw_stocks DataFrame
    for index, row in df_stocks.iterrows():
        ticker = row['Ticker']
        stock_name = tw.stock_name(ticker)
        # Update the stock name back to the DataFrame
        df_stocks.at[index, 'Ticker'] = stock_name

def industry_tickers_to_names(df_industries):
    # Iterate over the 'Tickers' column in the tw_industries DataFrame
    for index, row in df_industries.iterrows():
        tickers = row['Tickers'].split(',')  # Split the tickers string into a list
        stock_names = [tw.stock_name(ticker) for ticker in tickers]  # Get stock names for each ticker
        # Update the stock names back to the DataFrame
        df_industries.at[index, 'Tickers'] = ','.join(stock_names)  # Join the names back into a comma-separated string

filtered_tw_stocks = tw_stocks[
    (tw_stocks["Relative Strength"] > 100)
    & (tw_stocks["Relative Strength"] > tw_stocks["1 Month Ago"])
    & (tw_stocks["1 Month Ago"] > tw_stocks["3 Months Ago"])
    & (tw_stocks["3 Months Ago"] > tw_stocks["6 Months Ago"])
    #& (tw_stocks["Percentile"] > 90)
]
stock_ticker_to_name(filtered_tw_stocks)
display(filtered_tw_stocks)

filtered_tw_industries = tw_industries[
    (tw_industries["Relative Strength"] > 100)
    & (tw_industries["Relative Strength"] > tw_industries["1 Month Ago"])
    & (tw_industries["1 Month Ago"] > tw_industries["3 Months Ago"])
    & (tw_industries["3 Months Ago"] > tw_industries["6 Months Ago"])
    #& (tw_stocks["Percentile"] > 90)
]
industry_tickers_to_names(filtered_tw_industries)
display(filtered_tw_industries)

,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
207,得利影,Communication Services,Entertainment,470.91,254.34,119.00,83.36,99.0,99,87,15,1
1162,易威,Healthcare,Drug Manufacturers - Specialty & Generic,419.95,345.56,206.79,200.35,99.0,99,99,98,2
1530,穎漢,Industrials,Specialty Industrial Machinery,338.47,324.02,151.02,86.69,99.0,99,96,24,6
824,欣巴巴,Industrials,Engineering & Construction,284.74,152.72,121.08,114.58,99.0,96,89,82,11
1890,福懋油,Consumer Defensive,Packaged Foods,215.38,185.73,101.81,91.41,98.0,98,74,41,29
...,...,...,...,...,...,...,...,...,...,...,...,...
1474,美吉吉-KY,Industrials,Building Products & Equipment,103.46,101.35,101.00,100.74,75.0,75,73,64,491
881,東泥,Basic Materials,Building Materials,102.71,93.45,86.52,84.34,74.0,66,47,17,512
508,美隆電,Technology,Consumer Electronics,101.83,97.95,89.18,86.04,73.0,71,53,22,530
1675,亞泰,Technology,Electronic Components,100.52,90.33,90.03,89.89,71.0,60,55,35,572


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
18,Specialty Industrial Machinery,Industrials,120.06,112.92,105.04,104.94,"穎漢,福裕,高鋒,彬台,志聖,喬福,太普高,和椿,盟立,精湛,友威科,惠特,信紘科,時碩工業...",94.0,94,90,83,5
54,Security & Protection Services,Industrials,111.98,110.90,100.02,95.78,"慧友,京晨科,杭特,奇偶,中保科,福興,勝品,陞泰,新保,昇銳,邑錡,云辰,晶睿",89.0,93,84,63,9
60,Specialty Business Services,Industrials,109.78,103.98,99.50,94.87,"花王,沈氏,白紗科,秋雨,關貿,政伸,信實,耕興,遠雄港",87.0,87,82,54,10
